In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

%matplotlib inline
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

hp = pd.read_csv("~/NYCDSA/Machine_Learning_I/Machine_Learning_Project/Ames_Housing_ML/data/HousePrice1_no_outlier.csv", \
                 index_col = 0)
hp_res= hp[hp['MSZoning'].isin(['FV','RH','RL','RM'])]
hp_res = hp_res.dropna()
hp_res.shape

Current working directory: /Users/chereowang27/NYCDSA/Machine_Learning_I/Machine_Learning_Project/Ames_Housing_ML


(2563, 76)

In [7]:
hp_res1 = hp_res.drop(['MapRefNo','PID', 'Prop_Addr'], \
                               axis = 1)
num_features = hp_res1._get_numeric_data() 
num_features['bath'] = num_features['FullBath'] + 0.5*num_features['HalfBath']
numCol = num_features.columns

In [8]:
col = hp_res1.columns
catCol = [col for col in hp_res1.columns if col not in num_features.columns]
cat_features = hp_res[catCol]
cat_features.columns

Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition', 'mo_Yr_sold'],
      dtype='object')

In [10]:
features = pd.concat([num_features, cat_features], axis = 1) \
           .drop(['FullBath', 'HalfBath', \
                  'BsmtFullBath', 'BsmtHalfBath', \
                  'BsmtFinSF1','SalePrice','ExterQual', \
                  'Condition1','Condition2', \
                  'BsmtFinType1','BsmtFinType2', \
                  'BsmtQual', 'GarageQual','MoSold'],axis = 1)

In [12]:
# Feature Selection after lasso
features = features[['GrLivArea', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', \
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'TotalBsmtSF', '1stFlrSF', \
       '2ndFlrSF', 'BedroomAbvGr', 'KitchenAbvGr', \
       'Fireplaces', 'GarageYrBlt', 'GarageArea', \
       'WoodDeckSF','EnclosedPorch', '3SsnPorch',   \
       'ScreenPorch', 'MiscVal', 'YrSold',  \
       'MSZoning','Foundation', 'BsmtExposure',  \
       'HeatingQC','Neighborhood']]
catCol2 = ['Foundation', 'BsmtExposure', 'HeatingQC', 'MSZoning','Neighborhood'] 

features1= pd.get_dummies(features, columns = catCol2, drop_first = True)

y = hp_res['SalePrice']
hp_res['log_Price'] = np.log10(hp_res['SalePrice'])
logy = hp_res['log_Price']



In [21]:
from sklearn import linear_model, datasets, metrics
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import time
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(features1, logy, test_size = 0.3, random_state = 0)

In [22]:
#run multiple linear regression using features selected with Lasso regression
ols = LinearRegression()

ols.fit(X_train_log, y_train_log)

cv = np.mean(cross_val_score(ols, X_train_log, y_train_log))

pred = ols.predict(X_test_log)
RMSE = (mean_squared_error(pred, y_test_log, squared = False))


print("R^2 train set: %f" %ols3.score(X_train_log, y_train_log))
print("R^2 test  set: %f" %ols4.score(X_test_log, y_test_log))
print("CV R^2 train set: " + str(round(cv,5)))
print("RMSE: " + str(round(RMSE, 5))) 



R^2 train set: 0.929319
R^2 test  set: 0.931691
CV R^2 train set: 0.92297
RMSE: 0.04591


In [23]:
nb = features['Neighborhood'].unique()
def label_nb(row):
    if row['Neighborhood'] in (['NridgHt','NoRidge','Veenker','StoneBr']):      
        return 'N'
    if row['Neighborhood']      \
             in (['Blmngtn','Gilbert','OldTown', 'NWAmes','NAmes','Blmngtn','BrDale','Somerst','BrkSide','Greens','NPkVill']):
        return 'NE'
    if row['Neighborhood']      \
            in (['CollgCr','SWISU', 'Edwards','SawyerW','Sawyer','ClearCr']):
        return 'W'                      
    if row['Neighborhood'] in (['Crawfor','MeadowV','Timber','Mitchel','IDOTRR']):     
        return 'S'
    
features['Location']=features.apply(lambda row: label_nb(row), axis = 1)
features = features.drop(['Neighborhood'], axis = 1)
features2 = features    
catCol2 = ['Foundation', 'BsmtExposure', 'HeatingQC', 'MSZoning', 'Location'] 
features2= pd.get_dummies(features, columns = catCol2, drop_first = True)

In [80]:
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(features2, logy, test_size = 0.3, random_state = 0)

In [25]:
ols = LinearRegression()
ols.fit(X_train_log, y_train_log)
cv = np.mean(cross_val_score(ols, X_train_log, y_train_log))
pred = ols.predict(X_test_log)
RMSE = (mean_squared_error(pred, y_test_log, squared = False))


print("R^2 train set: %f" %ols.score(X_train_log, y_train_log))
print("R^2 test  set: %f" %ols.score(X_test_log, y_test_log))
print("CV R^2 train set: " + str(round(cv,5)))
print("RMSE: " + str(round(RMSE, 5))) 

R^2 train set: 0.929319
R^2 test  set: 0.920043
CV R^2 train set: 0.92297
RMSE: 0.04591
